In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# cd sample_data

/content/sample_data


In [4]:
# from google.colab import files
# upload = files.upload()

Saving glen.csv to glen.csv


In [5]:
data_set = pd.read_csv('glen.csv')

Dataset row column

In [6]:
data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4709 entries, 0 to 4708
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   DWD              4709 non-null   object
 1   name             4709 non-null   object
 2   wd_description   4709 non-null   object
 3   overlay_parents  4700 non-null   object
 4   Unnamed: 4       60 non-null     object
 5   Unnamed: 5       3 non-null      object
dtypes: object(6)
memory usage: 220.9+ KB


Find Nan values from entire dataset

In [7]:
# data_set.isnull().sum().sum()
# for cl in data_set.columns:
#   print(cl)
display(data_set.isnull().sum())


DWD                   0
name                  0
wd_description        0
overlay_parents       9
Unnamed: 4         4649
Unnamed: 5         4706
dtype: int64

Duplicate rows form entire dataset

In [8]:
# data_set[data_set['ID'].duplicated() == True]
print('duplicate row from entire dataset')
data_set[data_set.duplicated() == True]

duplicate row from entire dataset


,DWD,name,wd_description,overlay_parents,Unnamed: 4,Unnamed: 5


Duplicate row from Event name column

In [ ]:
print('duplicate row from event name column')
# dup_wd = data_set[data_set['name'].duplicated() == True]
# print(dup_wd.to_markdown())


duplicate row from event name column


In [9]:
data_set['name'].duplicated().value_counts()

False    4348
True      361
Name: name, dtype: int64

In [10]:
name_count = pd.value_counts(data_set.name)
name_count = data_set['name'].value_counts()
print(name_count)

# print(data_set.unique(data_set['name']))
# print(data_set.groupby('name').name)

flight                          5
motion                          5
competition                     4
order                           4
cancellation                    4
                               ..
epidemiological_surveillance    1
epidural_injection              1
epilation                       1
epileptic_seizure               1
public_pollution                1
Name: name, Length: 4348, dtype: int64


**One Hot Encoding**

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [15]:
data_encod = data_set
data_encod.DWD = le.fit_transform(data_encod.DWD)
data_encod.name = le.fit_transform(data_encod.name)
data_encod.wd_description = le.fit_transform(data_encod.wd_description)
data_encod.overlay_parents = le.fit_transform(data_encod.overlay_parents)
data_encod

,DWD,name,wd_description,overlay_parents,Unnamed: 4,Unnamed: 5
0,2735,392,3216,1053,NaN,NaN
1,1558,1924,1189,468,NaN,NaN
2,2731,1275,3785,1053,NaN,NaN
3,1090,704,254,1811,NaN,NaN
4,324,963,2927,79,NaN,NaN
...,...,...,...,...,...,...
4704,4606,1880,3775,802,NaN,NaN
4705,2129,1224,3341,1299,NaN,NaN
4706,2110,3955,4142,348,NaN,NaN
4707,600,4159,1249,1409,NaN,NaN


In [17]:
X = data_encod[['DWD','wd_description', 'overlay_parents']].values
X


array([[2735, 3216, 1053],
       [1558, 1189,  468],
       [2731, 3785, 1053],
       ...,
       [2110, 4142,  348],
       [ 600, 1249, 1409],
       [1107, 4352, 1725]])

In [18]:
y = data_encod.name.values
y

array([ 392, 1924, 1275, ..., 3955, 4159, 3162])

In [34]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
# ct = ColumnTransformer([(OneHotEncoder(), [0,1,2])], remainder = 'passthrough')
# ct = ColumnTransformer()
ct = ColumnTransformer(
                       [("imputer", SimpleImputer(missing_values=np.nan, strategy='median'), [2]),
                        ("one_hot", OneHotEncoder(categories='auto', sparse=False), slice(0, 1, 2))])

In [35]:
X = ct.fit_transform(X)
X

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[1053.,    0.,    0., ...,    0.,    0.,    0.],
       [ 468.,    0.,    0., ...,    0.,    0.,    0.],
       [1053.,    0.,    0., ...,    0.,    0.,    0.],
       ...,
       [ 348.,    0.,    0., ...,    0.,    0.,    0.],
       [1409.,    0.,    0., ...,    0.,    0.,    0.],
       [1725.,    0.,    0., ...,    0.,    0.,    0.]])

In [36]:
X = X[:,0:4]
X


array([[1053.,    0.,    0.,    0.],
       [ 468.,    0.,    0.,    0.],
       [1053.,    0.,    0.,    0.],
       ...,
       [ 348.,    0.,    0.,    0.],
       [1409.,    0.,    0.,    0.],
       [1725.,    0.,    0.,    0.]])

**Naive Bayes**

In [40]:
prior = data_encod.groupby("wd_description").size().div(len(data_encod))
print(prior)

wd_description
0       0.000212
1       0.000212
2       0.000212
3       0.000212
4       0.000212
          ...   
4539    0.000212
4540    0.000212
4541    0.000212
4542    0.000212
4543    0.000212
Length: 4544, dtype: float64


In [41]:
likelihood = {}
likelihood['name'] = data_encod.groupby(['wd_description', 'name']).size().div(len(data_encod)).div(prior)
print(likelihood)

{'name': wd_description  name
0               2730    1.0
1               2541    1.0
2               13      1.0
3               3891    1.0
4               108     1.0
                       ... 
4539            3128    1.0
4540            1458    1.0
4541            2540    1.0
4542            2719    1.0
4543            3296    1.0
Length: 4709, dtype: float64}


In [ ]:
# X = data_set.iloc[:, [2, 3]].values
# y = data_set.iloc[:, 1].values

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [43]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [44]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [45]:
y_pred  =  classifier.predict(X_test)

In [49]:
# y_pred
# y_test

In [53]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test, y_pred)
ac = accuracy_score(y_test,y_pred)
print(ac)
print(cm)

0.006369426751592357
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
